In [1]:
## import libraries and functions

import numpy as np
import matplotlib.pyplot as plt
import torch

device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'
dtype = torch.float64

arr_kwargs = {'dtype': dtype, 'device': device}

from generate_obs_and_truth import create_obs_op, generate_obs


In [ ]:
dt = 6*3600        # 6 hours


psi_long = torch.load('psi_long_run', weights_only = True)
psi_true = torch.load('psi_true', weights_only = True).to(device)

## observations
nl = 2
choice = 'satellite'
k = 0
delta_obs = 16
m = 300
s_obs = 2.0

seed = 0

nx = 128
ny = 128

H,obs_loc = create_obs_op(nx,ny,choice,m,delta_obs,k)


print('m is: ' + str(H.shape[0]))

m is: 344


## MF-EnKF experiments

In [ ]:
#########################################
## Forecast RMSE for ML vs LR
#########################################

from MF_EnKF import MF_EnKF_run

from localization import create_loc_mat

from initializeQG import generate_initial_ensemble

from baseline import baseline_run

torch.cuda.empty_cache()

K_da = 1500


# Inflation factor
alpha_X = 1.0
alpha_Uhat = alpha_X
alpha_U = 1.0

remove_bound = True

s_obs = 2.0

r_loc = 5

if remove_bound == True:
    nx = 128 - 2
    ny = 128 - 2
    H,obs_loc = create_obs_op(nx,ny,choice,m,delta_obs,k)
    y_obs = generate_obs(psi_true[:,:,1:-1,1:-1],s_obs,H,seed)
    rho_X = torch.load('loc_mat/rho_' + str(r_loc) + '_noBC.pt', weights_only = True).to(device)


else:
    nx = 128
    ny = 128
    H,obs_loc = create_obs_op(nx,ny,choice,m,delta_obs,k)
    y_obs = generate_obs(psi_true,s_obs,H,seed)
    rho_X = torch.load('loc_mat/rho_' + str(r_loc) + '.pt', weights_only = True).to(device)


mode = 'ML'         # one of {'LR', 'ML'}
r_low = 64          # lower-dimensional surrogate

M = 0

s_mod_X = 0.0
s_mod_U = 0.0

# try to adjust these without using inflation
recenter_analysis_list = ['ML'] # 'none', 'all' ,'control'
adjust_corr_list = [True]

enkf_type_list = ['det']
control_list = [True] # True, False

recenter_forecast = 'none'

recenter = True

nl = 2
nx = 128
ny = 128

seed = 0
N_large = 200

device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'
X_init = generate_initial_ensemble(psi_long, N_large,seed,device)

alpha_X_val = [1.0]
N_X_val = [25]
N_U_val = [50]

k_obs = 4

rmse_i = []
rmse_i_Vikram = []

P_Z_i = []
P_X_i = []


lambda_val = [0.5]

alpha_X = 1.0

show = 'mfenkf'


M = 10
seed_val = range(M)

rmse_1 = []

for lambda_ in lambda_val:
    for recenter_analysis in recenter_analysis_list:
        for adjust_corr in adjust_corr_list:
            for control in control_list:
                for enkf_type in enkf_type_list:
                    for N_X in N_X_val:
                        for N_U in N_U_val:

                            for seed in seed_val:

                                alpha_Uhat = alpha_X
                                alpha_U = alpha_X

                                y_obs = generate_obs(psi_true[:,:,1:-1,1:-1],s_obs,H,seed)

                                if show == 'mfenkf':
                                    try: 
                                        rmse_Z, Z, P_X, P_Uhat, P_U, P_Z, mu_X, mu_Uhat, mu_U, X, Uhat, U   = MF_EnKF_run(lambda_,N_X,N_U,s_obs,rho_X, alpha_X, alpha_Uhat, alpha_U, k_obs,y_obs,
                                    X_init, K_da, psi_true, mode, H,r_low,s_mod_X,s_mod_U,recenter_analysis, adjust_corr,control,enkf_type,recenter_forecast,
                                    dt, nl, nx, ny, remove_bound)
                                        rmse_i.append(rmse_Z[-1])


                                        A_X = 1/np.sqrt(N_X-1) * (X - mu_X[:,None] ).type(dtype)
                                        A_Uhat = 1/np.sqrt(N_X-1) * (Uhat - mu_Uhat[:,None] ).type(dtype)
                                        A_U = 1/np.sqrt(N_U-1) * (U - mu_U[:,None]).type(dtype)
                                        S_Z = A_X @ A_X.T + lambda_**2*(A_Uhat @ A_Uhat.T) + lambda_**2*(A_U @ A_U.T) - lambda_*(A_X @ A_Uhat.T) - lambda_*(A_Uhat @ A_X.T) 

                                        rmse_1.append(rmse_Z[-1])
                                        
                                        print('lambda is: ' + str(lambda_) + ', (N_X,N_U) is: (' + str(N_X) + ',' + str(N_U) + '), rmse MF-EnKF is: ' + str(rmse_Z[-1]), ', spread X: ' + str(P_X[-1]) + ', spread Z: ' + str(P_Z[-1].item()))
                                    except:
                                        print('Unstable')



                            print('Mean RMSE: for lambda = ' + str(lambda_) + ': ' + str(np.mean(rmse_1[-M:])) )

                          #  elif show == 'base':
                          #      try:
                           #       recenter = False
                            #      rmse_Z, rmse_X, Z,P_Z, P_X, P_U, X, U = baseline_run(N_X,N_U,s_obs,rho_X, alpha_X, k_obs,y_obs,
                             #       X_init, K_da, psi_true, mode, H, r_low, enkf_type, dt, remove_bound,recenter)
                            
                              #    rmse_i_basse.append(rmse_X[-1])

                               #   print('N_X,N_U is: ' + str(N_X) + ', ' + str(N_U) + ', rmse base is: ' + str(rmse_X[-1]))
                               # except:
                               #   rmse_i_base.append(np.nan)

                               #   print('N_X,N_U is: ' + str(N_X) + ', ' + str(N_U) + ', rmse base is: ' + str(np.nan))



lambda is: 0.5, (N_X,N_U) is: (25,50), rmse MF-EnKF is: 0.6062153978500201 , spread X: 0.40246144169878767, spread Z: 0.586284072458355
lambda is: 0.5, (N_X,N_U) is: (25,50), rmse MF-EnKF is: 0.6614956470073498 , spread X: 0.4187443770761782, spread Z: 0.6048207202386467
lambda is: 0.5, (N_X,N_U) is: (25,50), rmse MF-EnKF is: 0.6239138906669541 , spread X: 0.4096728749922268, spread Z: 0.5991741393985422
lambda is: 0.5, (N_X,N_U) is: (25,50), rmse MF-EnKF is: 0.621106637380238 , spread X: 0.40626803873567735, spread Z: 0.5908173308909859
lambda is: 0.5, (N_X,N_U) is: (25,50), rmse MF-EnKF is: 0.6561799707472593 , spread X: 0.41639653438818214, spread Z: 0.6093827048488031
lambda is: 0.5, (N_X,N_U) is: (25,50), rmse MF-EnKF is: 0.6597088904283154 , spread X: 0.4036571165708187, spread Z: 0.5974701895866282
lambda is: 0.5, (N_X,N_U) is: (25,50), rmse MF-EnKF is: 0.6191535095682477 , spread X: 0.4046958909651269, spread Z: 0.5824775054955007
lambda is: 0.5, (N_X,N_U) is: (25,50), rmse MF-

## EnKF experiments

In [13]:
from EnKF import EnKF_run

remove_boundary = True
alpha_X_val = [1.0]

N_X = 30

enkf_type = 'det'

mode = 'HR'

r_loc = 5
rho_X = torch.load('loc_mat/rho_' + str(r_loc) + '_noBC.pt', weights_only = True).to(device)

loc = True

seed = 1
y_obs = generate_obs(psi_true[:,:,1:-1,1:-1],s_obs,H,seed)

seed_val = range(10)

for alpha_X in alpha_X_val:
    for seed in seed_val:
        y_obs = generate_obs(psi_true[:,:,1:-1,1:-1],s_obs,H,seed)

        try:
            rmse_Z, Z_mfenkf, P_mfenkf, X_mfenkf = EnKF_run(N_X,s_obs,rho_X,alpha_X, k_obs,y_obs, X_init,K_da, psi_true,enkf_type,H,dt,remove_boundary,
             nl, nx,ny,mode,loc)
            print('..............................')
            print('alpha = ' + str(alpha_X) + ', RMSE: ' + str(rmse_Z[-1]))

        except:
            print('Unstable run')

RMSE is: 1.2942513083061555
RMSE is: 1.0928791771467252
RMSE is: 0.9864690669736544
RMSE is: 0.9036569375090134
RMSE is: 0.8398889767345163
RMSE is: 0.7988085906000998
RMSE is: 0.7611804198248164
RMSE is: 0.732806189189351
RMSE is: 0.7108045793933323
RMSE is: 0.6965039179126312
RMSE is: 0.6787315308356454
RMSE is: 0.6656523149731385
RMSE is: 0.654634479458311
..............................
alpha = 1.0, RMSE: 0.6479304575407077
RMSE is: 1.312738724769321
RMSE is: 1.0891563390651018
RMSE is: 0.97214312745204
RMSE is: 0.9047265556875388
RMSE is: 0.8640044241407216
RMSE is: 0.8163975450782872
RMSE is: 0.7755745144604415
RMSE is: 0.7467515629538898
RMSE is: 0.730651442846852
RMSE is: 0.7173139760855392
RMSE is: 0.7002776742513865
RMSE is: 0.6849774667746434
RMSE is: 0.6730536515862454
..............................
alpha = 1.0, RMSE: 0.6651633982576678
RMSE is: 1.225443114522299
RMSE is: 1.009510581827806
RMSE is: 0.9207779233412444
RMSE is: 0.8714465181243181
RMSE is: 0.8253733229277124
RM